In [ ]:
import pickle5 as pickle
with open('../../data/final_ents/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [6]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]
    if i == 10:
        break

0it [00:00, ?it/s]

In [9]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = True
if filtering == True:
    filtering_threshold = 0.8
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    checkpoint = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        if similarity.item() > filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import pandas as pd
pd.DataFrame(zip(selected_ents, texts), columns = ['ent', 'text']).to_csv('mini_corpus.csv', index=False)